In [1]:
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
from quantfreedom.enums import CandleBodyType
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.indicators.tv_indicators import sma_tv

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange="binance_us",
    symbol='BTCUSDT',
    timeframe='30m',
    candles_to_dl=1500,
)

In [3]:
timestamp = candles[:, CandleBodyType.Timestamp]
datetimes = timestamp.astype("datetime64[ms]")
open = candles[:, CandleBodyType.Open]
high = candles[:, CandleBodyType.High]
low = candles[:, CandleBodyType.Low]
close = candles[:, CandleBodyType.Close]

In [4]:
sma_fast = sma_tv(source=close, length=20)
sma_slow = sma_tv(source=close, length=60)

In [5]:
prev_sma_fast = np.roll(sma_fast, 1)
prev_sma_fast[0] = np.nan
prev_sma_slow = np.roll(sma_slow, 1)
prev_sma_slow[0] = np.nan

In [6]:
(prev_sma_fast < prev_sma_slow) & (sma_fast > sma_slow)

array([False, False, False, ..., False, False, False])

In [7]:
entries = (prev_sma_fast < prev_sma_slow) & (sma_fast > sma_slow)
cross_above_signal = np.where(entries, sma_fast, np.nan)

In [8]:
(prev_sma_fast > prev_sma_slow) & (sma_fast < sma_slow)

array([False, False, False, ..., False, False, False])

In [9]:
exits = (prev_sma_fast > prev_sma_slow) & (sma_fast < sma_slow)
cross_below_signal = np.where(exits, sma_fast, np.nan)
exit_prices = np.roll(np.where(exits, close, np.nan), 1)
exit_prices[0] = np.nan

In [14]:
go.Figure(
    data=[
        go.Candlestick(
            x=datetimes,
            open=open,
            high=high,
            low=low,
            close=close,
            name="Candles",
        ),
        go.Scatter(
            x=datetimes,
            y=sma_fast,
            name="SMA Fast",
            line_color="lightblue",
        ),
        go.Scatter(
            x=datetimes,
            y=sma_slow,
            name="SMA Slow",
            line_color="yellow",
        ),
        go.Scatter(
            x=datetimes,
            y=cross_above_signal,
            mode="markers",
            name="Entries",
            marker=dict(
                size=15,
                symbol="circle",
                color="#00F6FF",
                line=dict(
                    width=1,
                    color="DarkSlateGrey",
                ),
            ),
        ),
        go.Scatter(
            x=datetimes,
            y=cross_below_signal,
            mode="markers",
            name="Exits",
            marker=dict(
                size=15,
                symbol="triangle-up",
                color="#FF7B00",
                line=dict(
                    width=1,
                    color="DarkSlateGrey",
                ),
            ),
        ),
    ]
).update_layout(height=800, xaxis_rangeslider_visible=False)

()